In [9]:
import numpy as np
import nibabel
from scipy import ndimage
import os
from matplotlib.pyplot import imshow
from skimage import exposure
import skimage.io as io

from deepbrain import Extractor
import cv2 as cv

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
root = "./BrainS18/"
folders = ['1', '5', '7', '4', '148', '070', '14']
folders = [os.path.join(root, f) for f in folders]
print(folders)

['./BrainS18/1', './BrainS18/5', './BrainS18/7', './BrainS18/4', './BrainS18/148', './BrainS18/070', './BrainS18/14']


In [ ]:
ls 1/pre

In [ ]:
ls 1

In [11]:
def get_data(path):
    return nibabel.load(path).get_data()

def get_data_with_skull_scraping(path, PROB = 0.5):
    arr = nibabel.load(path).get_data()
    ext = Extractor()
    prob = ext.run(arr)
    mask = prob > PROB
    arr = arr*mask
    return arr

def histeq(data):
    print(type(data), data.shape)
    for slice_index in range(data.shape[2]):
        # data[:,:,slice_index]=cv.equalizeHist(data[:,:,slice_index])
        data[:,:,slice_index] = exposure.equalize_adapthist(data[:,:,slice_index].astype(np.int))
    return data

def to_uint8(data):
    data=data.astype(np.float)
    data[data<0]=0
    return ((data-data.min())*255.0/data.max()).astype(np.uint8)

def IR_to_uint8(data):
    data=data.astype(np.float)
    data[data<0]=0
    return ((data-800)*255.0/data.max()).astype(np.uint8)

In [12]:
def normalize_one_volume(volume):
    pixels = volume[volume>0]
    mean = pixels.mean()
    std  = pixels.std()
    volume = (volume - mean)/std
    mi = np.min(volume)
    ma = np.max(volume)
    #volume = (volume-mi)/(ma-mi)
    #out_random = np.random.normal(0, 1, size = volume.shape)
    #out[volume==0] = out_random[volume==0]
    return volume

# filenames = ['FLAIR.nii.gz', 'reg_T1.nii.gz', 'reg_IR.nii.gz']
filenames = ['reg_T1.nii.gz']
is_remove_skull = False
#if os.path.exists("train.txt"):
#    os.remove("train.txt")

#file_write_obj = open("train.txt", 'w')
#file_write_obj = open("vaild.txt", "w")
file_write_obj = open("data.txt", "w")

#folders = ['7', '4', '148', '070', '14']
#folders = ['5']
# folders = ['1']
folders = ['1', '5', '7', '4', '148', '070', '14']
folders = [os.path.join(root, f) for f in folders]

for folder in folders:
    flag = True
    for filename in filenames:
        full_path = folder+'/pre/'+filename

        if is_remove_skull:
            volume = get_data_with_skull_scraping(full_path, 0.3)
            # volume = histeq(volume)
            # volume = to_uint8(volume)
        else:
            volume = nibabel.load( full_path ).get_data()
        

        #volume = normalize_one_volume(volume)
        #volume = volume.astype('uint8')
        num_of_image = volume.shape[2]
        new_folder = folder.split("/")[2]
        if not os.path.exists(new_folder):
            os.mkdir(new_folder)
        for i in range(num_of_image):
            new_filename = str(i)+'_'+filename[:-7]+".png"
            new_id = str(i)
            #print(os.path.join(new_folder, new_filename))
            slice = volume[:,:,i]
            if not is_remove_skull:
                slice = exposure.equalize_adapthist(slice.astype(np.int))
            print(np.max(slice), np.min(slice))
            io.imsave(os.path.join(new_folder, new_filename), np.rot90(slice, k=-1)) # rorate 90 clockwise
            if flag:
                file_write_obj.writelines( new_folder+'/'+new_id )
                file_write_obj.write('\n')
        flag = False
            
        print(full_path)

file_write_obj.close()    

g.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
1.0 0.0
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior t

In [ ]:
folders = ['1','5', '7', '4', '148', '070', '14']
folders = [os.path.join(root, f) for f in folders]
filenames = ['segm.nii.gz']
print("----------------------")

for folder in folders:
    for filename in filenames:
        full_path = folder+'/'+filename
        volume = nibabel.load( full_path ).get_data()
        #volume = img.astype('uint8')
        num_of_image = volume.shape[2]
        new_folder = folder.split("/")[2]
        if not os.path.exists(new_folder):
            os.mkdir(new_folder)
        for i in range(num_of_image):
            new_filename = str(i)+'_'+filename[:-7]+".png"
            #print(os.path.join(new_folder, new_filename))
            io.imsave(os.path.join(new_folder, new_filename), np.rot90(volume[:,:,i], k=-1) ) # rorate 90 clockwise
        print(full_path)
        

In [ ]:
root = "/data/MICCAI_BraTS17_Data_Training_for_ENZhiying/MICCAI_BraTS17_Data_Training/HGG/"
patients = ["Brats17_CBICA_AAB_1","Brats17_CBICA_AOO_1"]
print(patients)

filenames = ['_flair.nii.gz', '_seg.nii.gz', '_t1.nii.gz' "_t1ce.nii.gz", "_t2.nii.gz"]

#folders = [os.path.join(root, f) for f in patients]
#print(folders)

for patient in patients:
    folder = os.path.join(root, patient)
    T1  = os.path.join(folder, patient+"_t1.nii.gz")
    seg = os.path.join(folder, patient+"_seg.nii.gz")
    health = "./BrainS18/1/pre/reg_T1.nii.gz"
    health_label = "./BrainS18/1/segm.nii.gz"

    T1  = nibabel.load( T1 ).get_data()
    seg = nibabel.load( seg ).get_data()
    health = nibabel.load( health ).get_data()
    health_label = nibabel.load( health_label ).get_data()
    
    #T1 = exposure.equalize_adapthist(T1)
    #health = exposure.equalize_adapthist(health)
    #T1     = normalize_one_volume(T1)
    #health = normalize_one_volume(health)
    
    mask = np.where(seg!=0, T1, 0)
    imshow(mask[:,:,75])
    imshow(seg[:,:,75])
    print(T1.shape)
    print(seg.shape)
    print(health.shape)
    num_of_image = health.shape[2]
    seg[seg==2]=0
    for i in range(num_of_image):
        #T1 = exposure.equalize_adapthist(T1)
        
        t1_slice = T1[:,:, 3*i]
        he_slice = health[:,:,i].astype(np.int)
        mi = np.min(t1_slice)
        ma = np.max(t1_slice)
        if mi==ma:
            t1_slice = np.zeros_like(t1_slice, dtype="uint8")
        else:
            t1_slice = exposure.equalize_adapthist(t1_slice)
        he_slice = exposure.equalize_adapthist(he_slice)
        print(np.min(t1_slice), np.max(t1_slice))
        combine = np.where(seg[:,:,3*i]!=0, t1_slice*0.8, he_slice)
        
        combine_label = np.where(seg[:,:, 3*i]!=0, 100, health_label[:,:,i])
        new_folder = patient + "_img"
        if not os.path.exists(new_folder):
            os.mkdir(new_folder)
        io.imsave(os.path.join(new_folder, str(i)+"_reg_T1.png"), np.rot90(combine))
        io.imsave(os.path.join(new_folder, str(i)+"_orgin.png"), np.rot90(t1_slice))
        io.imsave(os.path.join(new_folder, str(i)+"_segm.png"), np.rot90(combine_label))
        if i==25:
            #imshow(seg[:,:,3*i])
            imshow(combine)
